# <font color='blue'>Barnes, R. "Gene"</font> - <font color='purple'><u>HW Individual Tutorial: Image Classification</u></font> - 3/16/2018

## <font color='purple'>K-State Honor Code:</font> "On my honor, as a student, I have neither given nor received unauthorized aid on this academic work."

## In our current society, we are continually looking to improve security.
- Security is in many forms, cybersecurity (cryptography, servers, Internet), physical (access control, video survelliance, biometrics). We as humans have a need to improve on these as time progresses. In the past few years, and even the last several months, there have been incidents both on campus and across the nation that is fueling an urgency to improve security. 
- As I work with our security team in Housing and Dining Services, we too see a great need to improve our security for our residential students. Aside from upgrading an outdated video survelliance system, what if we were able to use Image Classification with our video survelliance to distinquish student, faculty and staff from those who aren't?

## Introducing Deep Learning and Convolutional Neural Networks (CNNs).
- First, what is "deep learning"? "Deep learning is a type of machine learning in which a model learns to
perform classification tasks directly from images, text, or sound. Deep
learning is usually implemented using a neural network architecture. The
term “deep” refers to the number of layers in the network—the more layers,
the deeper the network. Traditional neural networks contain only 2 or 3
layers, while deep networks can have hundreds."
- From https://www.mathworks.com/content/dam/mathworks/tag-team/Objects/d/80879v00_Deep_Learning_ebook.pdf

- Next, what is a CNN? "A convolutional neural network (CNN, or ConvNet) is one of the
most popular algorithms for deep learning with images and video.
Like other neural networks, a CNN is composed of an input layer,
an output layer, and many hidden layers in between.
Feature Detection Layers
These layers perform one of three types of operations on the data:
convolution, pooling, or rectified linear unit (ReLU).
Convolution puts the input images through a set of convolutional
filters, each of which activates certain features from the images.
Pooling simplifies the output by performing nonlinear
downsampling, reducing the number of parameters that the
network needs to learn about.
Rectified linear unit (ReLU) allows for faster and more effective
training by mapping negative values to zero and maintaining
positive values.
These three operations are repeated over tens or hundreds of
layers, with each layer learning to detect different features."
- From https://www.mathworks.com/content/dam/mathworks/tag-team/Objects/d/80879v00_Deep_Learning_ebook.pdf

## Using the Python library imgaug
- This library helps you with augmenting images for your machine learning projects. It converts a set of input images into a new, much larger set of slightly altered images.

## What is our goal?
- What we are looking at with this deep learning and the CNNs is taking images and using both, process the images by various means and sequences to ensure greater accuraccy with image classification.
- These sequences are how we get the process to see all parts of an image so that even one pixel or group of pixels may be used to determine a cat from a dog, or in our grand scheme, a residential student from staff. This could be anything from flipping images to blurring them, all to help increase the recognition and accuracy of one image from another.
- To meet our goal, or our example, we need to 
 1. Show how to use imgaug (Image Augmentation) sequence to help with determining images. 
 2. Show results of the examples. 
 3. 

### Python 2.7 coding for machine learning using imgaug

- Coding below here courtesy of https://github.com/aleju/imgaug
- Here this example shows an augmentation sequence that might be useful for many common experiments. It applies crops and affine transformations to images, flips some of the images horizontally, adds a bit of noise and blur and also changes the contrast as well as brightness. (http://imgaug.readthedocs.io/en/latest/source/examples_basics.html) 

In [1]:
import imgaug as ia
from imgaug import augmenters as iaa
import numpy as np

# random example images
images = np.random.randint(0, 255, (16, 128, 128, 3), dtype=np.uint8)

# Sometimes(0.5, ...) applies the given augmenter in 50% of all cases,
# e.g. Sometimes(0.5, GaussianBlur(0.3)) would blur roughly every second image.
sometimes = lambda aug: iaa.Sometimes(0.5, aug)

# Define our sequence of augmentation steps that will be applied to every image
# All augmenters with per_channel=0.5 will sample one value _per image_
# in 50% of all cases. In all other cases they will sample new values
# _per channel_.
seq = iaa.Sequential(
    [
        # apply the following augmenters to most images
        iaa.Fliplr(0.5), # horizontally flip 50% of all images
        iaa.Flipud(0.2), # vertically flip 20% of all images
        # crop images by -5% to 10% of their height/width
        sometimes(iaa.CropAndPad(
            percent=(-0.05, 0.1),
            pad_mode=ia.ALL,
            pad_cval=(0, 255)
        )),
        sometimes(iaa.Affine(
            scale={"x": (0.8, 1.2), "y": (0.8, 1.2)}, # scale images to 80-120% of their size, individually per axis
            translate_percent={"x": (-0.2, 0.2), "y": (-0.2, 0.2)}, # translate by -20 to +20 percent (per axis)
            rotate=(-45, 45), # rotate by -45 to +45 degrees
            shear=(-16, 16), # shear by -16 to +16 degrees
            order=[0, 1], # use nearest neighbour or bilinear interpolation (fast)
            cval=(0, 255), # if mode is constant, use a cval between 0 and 255
            mode=ia.ALL # use any of scikit-image's warping modes (see 2nd image from the top for examples)
        )),
        # execute 0 to 5 of the following (less important) augmenters per image
        # don't execute all of them, as that would often be way too strong
        iaa.SomeOf((0, 5),
            [
                sometimes(iaa.Superpixels(p_replace=(0, 1.0), n_segments=(20, 200))), # convert images into their superpixel representation
                iaa.OneOf([
                    iaa.GaussianBlur((0, 3.0)), # blur images with a sigma between 0 and 3.0
                    iaa.AverageBlur(k=(2, 7)), # blur image using local means with kernel sizes between 2 and 7
                    iaa.MedianBlur(k=(3, 11)), # blur image using local medians with kernel sizes between 2 and 7
                ]),
                iaa.Sharpen(alpha=(0, 1.0), lightness=(0.75, 1.5)), # sharpen images
                iaa.Emboss(alpha=(0, 1.0), strength=(0, 2.0)), # emboss images
                # search either for all edges or for directed edges,
                # blend the result with the original image using a blobby mask
                iaa.SimplexNoiseAlpha(iaa.OneOf([
                    iaa.EdgeDetect(alpha=(0.5, 1.0)),
                    iaa.DirectedEdgeDetect(alpha=(0.5, 1.0), direction=(0.0, 1.0)),
                ])),
                iaa.AdditiveGaussianNoise(loc=0, scale=(0.0, 0.05*255), per_channel=0.5), # add gaussian noise to images
                iaa.OneOf([
                    iaa.Dropout((0.01, 0.1), per_channel=0.5), # randomly remove up to 10% of the pixels
                    iaa.CoarseDropout((0.03, 0.15), size_percent=(0.02, 0.05), per_channel=0.2),
                ]),
                iaa.Invert(0.05, per_channel=True), # invert color channels
                iaa.Add((-10, 10), per_channel=0.5), # change brightness of images (by -10 to 10 of original value)
                iaa.AddToHueAndSaturation((-20, 20)), # change hue and saturation
                # either change the brightness of the whole image (sometimes
                # per channel) or change the brightness of subareas
                iaa.OneOf([
                    iaa.Multiply((0.5, 1.5), per_channel=0.5),
                    iaa.FrequencyNoiseAlpha(
                        exponent=(-4, 0),
                        first=iaa.Multiply((0.5, 1.5), per_channel=True),
                        second=iaa.ContrastNormalization((0.5, 2.0))
                    )
                ]),
                iaa.ContrastNormalization((0.5, 2.0), per_channel=0.5), # improve or worsen the contrast
                iaa.Grayscale(alpha=(0.0, 1.0)),
                sometimes(iaa.ElasticTransformation(alpha=(0.5, 3.5), sigma=0.25)), # move pixels locally around (with random strengths)
                sometimes(iaa.PiecewiseAffine(scale=(0.01, 0.05))), # sometimes move parts of the image around
                sometimes(iaa.PerspectiveTransform(scale=(0.01, 0.1)))
            ],
            random_order=True
        )
    ],
    random_order=True
)

images_aug = seq.augment_images(images)

- Quickly show example results of your augmentation sequence:

In [2]:
from imgaug import augmenters as iaa
import numpy as np

images = np.random.randint(0, 255, (16, 128, 128, 3), dtype=np.uint8)
seq = iaa.Sequential([iaa.Fliplr(0.5), iaa.GaussianBlur((0, 3.0))])

# show an image with 8*8 augmented versions of image 0
seq.show_grid(images[0], cols=8, rows=8)

# Show an image with 8*8 augmented versions of image 0 and 8*8 augmented
# versions of image 1. The identical augmentations will be applied to
# image 0 and 1.
seq.show_grid([images[0], images[1]], cols=8, rows=8)

RuntimeError: Could not execute image viewer.